In [16]:
# library
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import LSTM

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

# seed


In [3]:
# load dataset
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255
x_test = x_test / 255

x_dnn = x_train.reshape(-1, 784)
print(x_dnn.shape)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [18]:
# PCA
pca = PCA()
pca.fit(x_dnn)
cumsum = np.cumsum(pca.explained_variance_ratio_)
print(cumsum)

n = np.argmax(cumsum >= 0.99) + 1
print(n)

[0.09704664 0.16800588 0.22969677 0.28359097 0.33227894 0.37540125
 0.40812055 0.4369595  0.4645798  0.4881498  0.5092417  0.52947161
 0.54662979 0.56355091 0.57933732 0.59416685 0.60741246 0.62018143
 0.63205406 0.6435809  0.65424256 0.66430969 0.67384542 0.68297086
 0.69180491 0.7001981  0.70832389 0.71618755 0.72363488 0.73054347
 0.73712441 0.74360589 0.74963204 0.75549786 0.76119807 0.76663435
 0.77169222 0.7765708  0.7813851  0.78610776 0.79067523 0.79512359
 0.7993086  0.80329076 0.8071405  0.81089154 0.81451162 0.81802754
 0.82142812 0.82464686 0.82783703 0.83096508 0.83392491 0.83681446
 0.83965576 0.84237012 0.84506533 0.84765006 0.85018776 0.85263557
 0.85504063 0.85743326 0.85973735 0.86195267 0.86408988 0.86616213
 0.86819256 0.87016038 0.87208891 0.87397523 0.875845   0.87765583
 0.87943146 0.88118044 0.88283802 0.88447696 0.88609158 0.88764274
 0.88911887 0.89055063 0.89197158 0.89338311 0.89478485 0.89614222
 0.89748069 0.89880465 0.90010622 0.90136495 0.90259324 0.9038

In [5]:
# model
def DNN_model(drop=0.2, optimizer='adam'):
    inputs = Input(shape=(784))
    l1 = Dense(1000, activation='selu')(inputs)
    l1 = Dropout(drop)(l1)
    l2 = Dense(500, activation='selu')(l1)
    l2 = Dropout(drop)(l2)
    l3 = Dense(100, activation='selu')(l2)
    l3 = Dropout(drop)(l3)
    outputs = Dense(1)(l3)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = optimizer, loss='mse')
    return model

def CNN_model(drop=0.2, optimizer='adam'):
    inputs = Input(shape=(28,28,1))
    l1 = Conv2D(32, (3,3), activation='selu', padding='same', kernel_initializer='he_normal')(inputs)
    l1 = Conv2D(32, (3,3), activation='selu', padding='same', kernel_initializer='he_normal')(l1)
    l1 = BatchNormalization()(l1)
    l1 = Activation('selu')(l1)
    l1 = MaxPooling2D((2,2))(l1)
    l2 = Conv2D(64, (3,3), activation='selu', padding='same', kernel_initializer='he_normal')(l1)
    l2 = MaxPooling2D((2,2))(l2)
    l3 = Flatten()(l2)
    l4 = Dense(100, activation='selu')(l3)
    l4 = Dropout(drop)(l4)
    outputs = Dense(1)(l4)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = optimizer, loss='mse')
    return model

def LSTM_model(drop=0.2, optimizer='adam'):
    inputs = Input(shape=(28,28))
    l1 = LSTM(256, activation='selu', return_sequences=True)(inputs)
    l1 = Dropout(drop)(l1)
    l2 = LSTM(128, activation='selu')(l1)
    l2 = Dropout(drop)(l2)
    l3 = Dense(100, activation='selu')(l2)
    l3 = Dropout(drop)(l3)
    outputs = Dense(1)(l3)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = optimizer, loss='mse')
    return model

In [19]:
# hyperparameter
def hyperparameters():
    dropout = [0.1, 0.2, 0.3, 0.4, 0.5]
    optimizers = ['adam', 'adadelta', 'rmsprop']
    batches = [64, 128, 256]
    epochs = [30, 50, 100]
    return {'drop':dropout, 'optimizer':optimizers, 'batch_size':batches,
            'epochs':epochs}
hyperparameters = hyperparameters()

In [20]:
# random search
DNN = KerasRegressor(build_fn=DNN_model, verbose=1)
#CNN = KerasRegressor(build_fn=CNN_model, verbose=1)
#LSTM = KerasRegressor(build_fn=LSTM_model, verbose=1)
best_DNN = RandomizedSearchCV(DNN, hyperparameters, cv=5)
#best_CNN = RandomizedSearchCV(CNN, hyperparameters, cv=5)
#best_LSTM = RandomizedSearchCV(LSTM, hyperparameters, cv=5)

In [21]:
# training
best_DNN.fit(x_dnn, y_train)
print(best_DNN.best_params_)

Train on 48000 samples
Epoch 1/30
48000/48000 [==============================] - 8s 163us/sample - loss: 7.0356
Epoch 2/30
48000/48000 [==============================] - 7s 144us/sample - loss: 4.0372
Epoch 3/30
48000/48000 [==============================] - 7s 147us/sample - loss: 2.8965
Epoch 4/30
48000/48000 [==============================] - 7s 155us/sample - loss: 2.3730
Epoch 5/30
48000/48000 [==============================] - 7s 153us/sample - loss: 2.1211
Epoch 6/30
48000/48000 [==============================] - 7s 153us/sample - loss: 1.9407
Epoch 7/30
48000/48000 [==============================] - 7s 153us/sample - loss: 1.8313
Epoch 8/30
48000/48000 [==============================] - 7s 153us/sample - loss: 1.6997
Epoch 9/30
48000/48000 [==============================] - 7s 154us/sample - loss: 1.6118
Epoch 10/30
48000/48000 [==============================] - 7s 156us/sample - loss: 1.5488
Epoch 11/30
48000/48000 [==============================] - 8s 157us/sample - loss: 1.5

KeyboardInterrupt: 